# PREDICT USING PRETRAINED MODEL #

#### GPU AND LIBRARIES ####

In [ ]:
!nvidia-smi

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

__author__ = 'Petros'

my_seed = 1989
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
import os
import sys
import numpy as np
import pickle
import torch.backends.cudnn as cudnn
import random
import json
import copy
from pytorch_pretrained_bert import BertTokenizer, BertModel
from tqdm import tqdm
from nltk import sent_tokenize

random.seed(my_seed)
torch.manual_seed(my_seed)
np.random.seed(my_seed)

cudnn.benchmark = True

embedding_dim = 30
hidden_dim = 100
gpu_device = 0
use_cuda = torch.cuda.is_available()
if (use_cuda):
    torch.cuda.manual_seed(my_seed)
    print("Using GPU")


#### DECLARATION OF FUNCTIONS, CLASS AND METHODS ####

In [ ]:
#MAIN FUNCTIONS
def print_params(show_model=True):
    if show_model:
        print(40 * '=')
        print(model)
    print(40 * '=')
    total_params = 0
    print('Trainable Parameters\n')
    for parameter in model.parameters():
        if parameter.requires_grad:
            v = 1
            for s in parameter.size():
                v *= s
            total_params += v
    print(40 * '=')
    print(total_params)
    print(40 * '=')


class SciBertReaderSum(nn.Module):
    def __init__(self, frozen_top):
        super(SciBertReaderSum, self).__init__()

        self.tok = BertTokenizer.from_pretrained('./biomrc/scibert_scivocab_uncased/')
        self.bert = BertModel.from_pretrained('./biomrc/scibert_scivocab_uncased/')
        self.linear = nn.Linear(2 * 768, 100, bias=True)
        self.linear2 = nn.Linear(100, 1, bias=True)

        self.frozen_top = frozen_top

        for p in self.bert.parameters():
            p.requires_grad = False

        if use_cuda:
            self.bert = self.bert.cuda(gpu_device)
            self.linear = self.linear.cuda(gpu_device)
            self.linear2 = self.linear2.cuda(gpu_device)

    def freeze_top(self, optim):
        for p in self.bert.encoder.layer[-1].parameters():
            p.requires_grad = False
        for g in optim.param_groups:
            g['lr'] = 0.001
        self.frozen_top = True

    def unfreeze_top(self, optim):
        for p in self.bert.encoder.layer[-1].parameters():
            p.requires_grad = True
        for g in optim.param_groups:
            g['lr'] = 0.0001
        self.frozen_top = False

    def fix_input(self, abstract, title):
        ab_sents = sent_tokenize(abstract, 'english')
        ab_tok = [self.tok.tokenize(ab) for ab in ab_sents]
        entity_indices = list()
        entity_texts = list()
        for i in range(len(ab_tok)):
            ab_tok[i].insert(0, '[CLS]')
            entity_indices.append(list())
            entity_texts.append(list())
            for j in range(len(ab_tok[i])):
                if ab_tok[i][j] == '@':
                    if ab_tok[i][j+1] == 'entity':
                        entity_indices[-1].append(j)
            for j in entity_indices[-1]:
                n = j+1
                while ab_tok[i][n].startswith('##') or ab_tok[i][n] == 'entity':
                    n += 1
                    if n >= len(ab_tok[i]):
                        break
                entity_texts[-1].append(''.join(ab_tok[i][j:n]).replace('##', ''))
        ti_tok = self.tok.tokenize(title.replace('.XXXX', ' [MASK]').replace('XXXX', '[MASK]'))
        ti_tok.insert(0, '[SEP]')
        combined = list()
        mask_indices = list()
        for i in range(len(ab_tok)):
            combined.append(list())
            combined[i].extend(ab_tok[i])
            combined[i].extend(ti_tok)
            mask_indices.append(combined[i].index('[MASK]'))
        combined_inp = [torch.LongTensor(self.tok.convert_tokens_to_ids(c)).unsqueeze(dim=0) for c in combined]

        if use_cuda:
            combined_inp = [e.cuda(gpu_device) for e in combined_inp]

        return combined_inp, mask_indices, entity_indices, entity_texts

    def forward(self, abstract, title, entity_list, answer, ignore_big=True):
        combined_input, mask_indices, entity_indices, entity_texts = self.fix_input(abstract, title)
        max_len = max([e.shape[-1] for e in combined_input])
        if ignore_big and max_len > 512:
            return None, None
        # Pad combined input
        combined_input = torch.stack([F.pad(e, [0, max_len - e.shape[-1], 0, 0], 'constant', 0) for e in combined_input], dim=0).squeeze(dim=1)
        if use_cuda:
            combined_input = combined_input.cuda(gpu_device)
        out = list()
        bert_out = self.bert(combined_input)[0][-1]
        for i in range(combined_input.shape[0]):
            if len(entity_indices[i]) == 0:
                continue
            if len(entity_indices[i]) != 1:
                bert_out_entities = bert_out[i, entity_indices[i], :].squeeze(dim=0)
            else:
                bert_out_entities = bert_out[i, entity_indices[i], :]
            bert_out_mask = bert_out[i, mask_indices[i], :]
            bert_out_mask = bert_out_mask.expand_as(bert_out_entities)
            bert_out_concat = torch.cat([bert_out_entities, bert_out_mask], dim=-1)
            out.append(self.linear2(F.relu(self.linear(bert_out_concat))))
        entity_texts = [e for e in entity_texts if len(e) != 0]
        # Predict also
        preds = dict()
        for et, ot in zip(entity_texts, out):
            ot = ot.detach().cpu().numpy().tolist()
            for e, o in zip(et, ot):
                if e not in preds:
                    preds[e] = 0
                preds[e] += o[0]
        entity_outs = dict()
        for ent in entity_list:
            entity_outs[ent] = list()
            for r_i, r in enumerate(entity_texts):
                for c_i, c in enumerate(r):
                    if c == ent:
                        entity_outs[ent].append(out[r_i][c_i])
        for ent in list(entity_outs):
            if len(entity_outs[ent]) == 0:
                del entity_outs[ent]
            else:
                entity_outs[ent] = torch.sigmoid(torch.sum(torch.cat(entity_outs[ent])))
        return torch.stack(list(entity_outs.values())), entity_outs.keys(), preds

    def predict(self, abstract, title, entity_list, ignore_big=True):
        combined_input, mask_indices, entity_indices, entity_texts = self.fix_input(abstract, title)
        max_len = max([e.shape[-1] for e in combined_input])
        if ignore_big and max_len > 512:
            return None
        # Pad combined input
        combined_input = torch.stack(
            [F.pad(e, [0, max_len - e.shape[-1], 0, 0], 'constant', 0) for e in combined_input], dim=0).squeeze(dim=1)
        if use_cuda:
            combined_input = combined_input.cuda(gpu_device)
        out = list()
        bert_out = self.bert(combined_input)[0][-1]
        for i in range(combined_input.shape[0]):
            if len(entity_indices[i]) == 0:
                continue
            if len(entity_indices[i]) != 1:
                bert_out_entities = bert_out[i, entity_indices[i], :].squeeze(dim=0)
            else:
                bert_out_entities = bert_out[i, entity_indices[i], :]
            bert_out_mask = bert_out[i, mask_indices[i], :]
            bert_out_mask = bert_out_mask.expand_as(bert_out_entities)
            bert_out_concat = torch.cat([bert_out_entities, bert_out_mask], dim=-1)
            out.append(self.linear2(F.relu(self.linear(bert_out_concat))))
        # Find predictions
        entity_texts = [e for e in entity_texts if len(e)!= 0]
        preds = dict()
        for et, ot in zip(entity_texts, out):
            ot = ot.detach().cpu().numpy().tolist()
            for e, o in zip(et, ot):
                if e not in preds:
                    preds[e] = 0
                preds[e] += o[0]
        return preds

#### LOAD CHECKPOINT OF PRETRAINED MODEL ####

In [ ]:
# Load checkpoint
resume_from = './biomrc/model/scibertreadersum_best_checkpoint.pth.tar' #'./bert_model_results/scibertreadersum_checkpoint.pth.tar'
resumed = False
if os.path.exists(resume_from):
    checkpoint = torch.load(resume_from, map_location=torch.device('cpu')) # checkpoint = torch.load(resume_from)
    start_epoch = checkpoint['epoch'] + 1
    best_dev_acc = checkpoint['best_acc']
    best_epoch = checkpoint['best_epoch']
    frozen_t = checkpoint['frozen_top']
    early_stop_counter = checkpoint['early_stop']
    print("=> loaded checkpoint '{}' (epoch {})".format(resume_from, checkpoint['epoch']))
    print([(e, checkpoint[e]) for e in checkpoint.keys() if e!= 'state_dict' and e!='optimizer'])
    resumed = True
else:
    print('No checkpoint to load!')
    start_epoch = 0
    best_dev_acc = -1
    best_epoch = -1
    early_stop_counter = 0
    frozen_t = True


model = SciBertReaderSum(frozen_t)

if use_cuda:
    model.cuda(gpu_device)

cross_entropy = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print_params()

In [ ]:
#max_epochs = 40

# Try to resume
if resumed:
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    # Unfreeze top bert layer if we resumed with unfrozen top
    if not model.frozen_top:
        model.unfreeze_top(optimizer)
        print('BERT Top Layer is Unfrozen')

#### LOAD DATASET ####

In [ ]:
from datasets import load_dataset
# Load biomrc dataset
dataset = load_dataset('biomrc', 'biomrc_small_B')
print(dataset)

In [ ]:
dataset['test'][0]['abstract']

#### SHOW EXTRACT OF DATASET ####

In [ ]:
start_index = 0
end_index = 1
for n in range(start_index, end_index):
    print(f"dataset['test'][{n}]:\n{dataset['test'][n]}\n")
    print(f"abstract: \n{dataset['test'][n]['abstract']}")
    print(f"title: \n{dataset['test'][n]['title']}")
    print(f"entities_list: \n{dataset['test'][n]['entities_list']}\n")
    print(f"answer: \n{dataset['test'][n]['answer']}")
    print("-" * 40)

#### TEST MODEL USING DATASET ####

In [ ]:
n = 0
ab = dataset['test'][n]['abstract']
ti = dataset['test'][n]['title']
ents = dataset['test'][n]['entities_list']

In [ ]:
# OWN EDITED PREDICTION
preds = model.predict(ab, ti, ents)
print(f"Prediction: {preds}")
# Maximum value
best_prediction = max(preds, key=preds.get)

# Best prediction
print(f"Best Prediction: {best_prediction}")

#### PREPROCESSING OF DATA BEFORE USE MODEL ####

In [ ]:
#PREPROCESSING BEFORE RUN THE MODEL
import scispacy
import spacy
from spacy import displacy

#nlp1 = spacy.load("en_core_web_sm")
nlp2 = spacy.load("en_ner_bc5cdr_md")
#med7 = spacy.load("en_core_med7_lg")

# create distinct colours for labels
col_dict = {}
seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

In [ ]:
#question = "Has the pt. ever been on caltrate plus d2 tablets before"
question = "Has the patient is a 76-year-old female with a history of mitral regurgitation ever been on XXXX plus d2 tablets before"
context = "The patient is a 76-year-old female with a history of mitral regurgitation, congestive heart failure, recurrent UTIs, and uterine prolapse who presented with chills and hypotension and was admitted to the Medical ICU for treatment of septic shock. Mean arterial pressures were kept above 65 with Levophed and antibiotics were changed to penicillin 3 million units IV q.4h. and gentamicin 50 mg IV q.8h. An ATEE on 10/19 showed severe mitral regurgitation with posterior leaflet calcifications and linear density concerning for endocarditis, for which a PICC line was placed on 1/19 for a six-week course of penicillin 3 million units IV q.4h. and two-week course of gentamicin 50 mg IV q.8h. until 2/25. The patient was initially treated with Levophed for her hypotension until 11/0, and was placed on Levofloxacin and Vancomycin to treat Gram-positive cocci bacteremia and UTI. She was maintained on telemetry and was found to be a normal sinus rhythm with ectopy, including short once of nonsustained ventricular tachycardia. She was started on Lopressor 12.5 mg t.i.d. on 3/18, and this was increased to 25 mg b.i.d. at discharge, with her heart rates continuing to be between the 70s and the 90s, however, with less episodes of ectopy. Aspirin was given, and Lipitor was initially held for an initial transaminitis presumed to be secondary to shock liver. She had guaiac positive stools in the medical ICU, her hematocrit was stable around 33%, and her iron studies suggested anemia of chronic disease with possibly overlying iron deficiency. She had a normal random cortisol level of 35.3, and her Hemoglobin A1c was 6.5, so she was maintained thereafter only on insulin sliding scale and rarely required any coverage. The patient was kept on Lovenox and Protonix and her DISCHARGE MEDICATIONS include Aspirin 81 mg daily, iron sulfate 325 mg daily, gentamicin sulfate 50 mg IV q.8h. until 2/25 for a two-week course, penicillin G potassium 3 million units IV q.4h. until 0/12 for a six-week course, Lopressor 25 mg b.i.d., Caltrate plus D2 tablets p.o. daily, Lipitor 10 mg daily, and Protonix 40 mg daily. She was discharged to rehabilitation at Acanmingpeerra Virg Tantblu Medical Center in order to be able to get her antibiotic therapy, and her physicians will attempt to add the ACE back onto her medical regimen for better afterload reduction as her blood pressure tolerates, and potentially they will add her back on to the Lasix as well. She will require weekly lab draws to check her electrolytes and CBC while she is on the antibiotics."

In [ ]:
question = "Has the patient Peter, with a history of mitral regurgitation, XXXX"
context = "Peter is a patient with a history of mitral regurgitation and he has lung cancer"

In [ ]:
#context1 = nlp1(context)
context_ner = nlp2(context)
question_ner = nlp2(question)
#context3 = med7(context)

In [ ]:
#ENTITY CLASSIFICATION in CONTEXT GROUPING REPEATED ONES IN ORDER TO COUNT AND ASSIGN A LABEL
entities_context = []
entities_texts = []
count = 0
for ent in context_ner.ents:
    #print(f"Texto completo: {context2.text[ent.start_char:ent.end_char]}\n")
    if ent.text not in entities_texts:
        entity_item = {'label': f"@entity{count}", 'text': ent.text, 'entity': ent.label_, 'positions': [{'start_char': ent.start_char, 'end_char': ent.end_char}]}    
        entities_context.append(entity_item)
        entities_texts.append(ent.text)
        count+=1
    else:
        index = entities_texts.index(ent.text)
        entities_context[index]['positions'].append({'start_char': ent.start_char, 'end_char': ent.end_char})

#REPLACING IN THE CONTEXT
for item in entities_context:
    context = context.lower().replace(item['text'].lower(), item['label'])
    #for pos in item['positions']:
        #context = context[:pos['start_char']] + item['label'] + context[pos['end_char']:]



#ENTITY CLASSIFICATION IN QUESTION GROUPING REPEATED ONES IN ORDER TO COUNT AND ASSIGN A LABEL
entities_question = []
count = 0
for ent in question_ner.ents:
    if ent.text not in entities_texts:
        entity_item = {'label': f"@entity{count}", 'text': ent.text, 'entity': ent.label_}    
        entities_context.append(entity_item)
        count+=1      

entity_list = []        
#REPLACING IN THE CONTEXT
for item in entities_context:
    question = question.replace(item['text'].lower(), item['label'])
    entity_list.append(item['label'])

In [ ]:
print(f"Context: \n{context}\n\nQuestion: {question}\n\entity_list: {entity_list}\n\entities_context: {entities_context}")
print("-" * 160)

#### PREDICTION ####

In [ ]:
# PREDICTION
preds = model.predict(context, question, entity_list)
print(f"Prediction: {preds}")
# Maximum value
best_prediction = max(preds, key=preds.get)

for item in entities_context:
    if item['label'] == best_prediction:
        value_entity = item['text']
        break
    
# Best prediction
print(f"\nBEST PREDICTION: {best_prediction}, VALUE IS: {value_entity}")

## METRICS ##

#### LOAD DATASET TO TEST ####

In [ ]:
# Load biomrc_small_A dataset to test model
dataset_small_A = load_dataset('biomrc','biomrc_small_A')
print(dataset_small_A)

#### METHODS AND FUNCTIONS TO GET INFORMATION ####

In [ ]:
import ast
def get_true_answer(raw_true_answer):
    true_answer_string = raw_true_answer.split("::")[2].split("\n")[0].strip() #get answer
    true_label = raw_true_answer.split("::")[0].split("\n")[0].strip() #get label
    try:
        true_answer_list = ast.literal_eval(true_answer_string)
        if not isinstance(true_answer_list, list):
            print("The string isn't a list.")
        else:
            true_answer = true_answer_list[0].lower()
    except (SyntaxError, ValueError) as e:
        print(f"Error to eval string: {e}")
        true_answer = None
    return true_answer, true_label
    

def get_entities_list(list):
    entities_list = []
    entities_and_text_list = []
    for item in list:
        entity = item.split("::")[0].split("\n")[0].strip() #get entity
        text_raw = item.split("::")[2].split("\n")[0].strip() #get text
        textlist = ast.literal_eval(text_raw)
        text = textlist[0].lower()      
        entities_list.append(entity) 
        entities_and_text_list.append({'label':entity,'text':text})
    return entities_list, entities_and_text_list

def get_predict(context, question, entities_list, entities_and_text_list):
    preds = model.predict(context, question, entities_list)
    best_prediction = max(preds, key=preds.get) #Maximum value
    for item in entities_and_text_list:
        if item['label'] == best_prediction:
            value_entity = item['text']
            break
    return best_prediction, value_entity

In [ ]:
n = 12
label = 'test'#'validation'#'train'
print(f"dataset_small_A[{label}][{n}]:\n{dataset_small_A[label][n]}\n")
print(f"abstract: \n{dataset_small_A[label][n]['abstract']}")
print(f"title: \n{dataset_small_A[label][n]['title']}")
print(f"entities_list: \n{dataset_small_A['test'][n]['entities_list']}\n")
print(f"answer: \n{dataset_small_A[label][n]['answer']}")
print("-" * 40)

#### TEST FUNCTIONS AND METHODS ####

In [ ]:
true_answer, true_label = get_true_answer(dataset_small_A[label][n]['answer'])
print(f"True answer: {true_answer}\n")
print(f"Answer: {dataset_small_A[label][n]['answer']}\n")
entities_list, entities_and_text_list = get_entities_list(dataset_small_A[label][n]['entities_list'])
print(f"Entities list:\n{entities_list}\n")
print(f"Entities list and texts:\n{entities_and_text_list}\n")
context = dataset_small_A[label][n]['abstract']
question = dataset_small_A[label][n]['title']
entities_list, entities_and_text_list = get_entities_list(dataset_small_A[label][n]['entities_list'])
# PREDICTION
predicted_answer, predicted_label = get_predict(context, question, entities_list, entities_and_text_list)
print(predicted_answer)
print(predicted_label)

#### GET GROUND TRUE AND PREDICTED ANSWERS OF DATASET ####

In [ ]:
from IPython.display import clear_output
count = 0
answers_list = []
for item in dataset_small_A[label]:
    true_answer, true_label = get_true_answer(item['answer'])
    if true_answer != None:    
        entities_list, entities_and_text_list = get_entities_list(item['entities_list'])
        predicted_label, predicted_answer = get_predict(item['abstract'], item['title'], entities_list)
        data_dictionary = {'abstract': item['abstract'], 'title': item['title'], 'entities_list': item['entities_list'], 
                           'entities_and_text_list': entities_and_text_list, 'true_answer': true_answer, 'true_label': true_label, 
                           'predicted_answer': predicted_answer, 'predicted_label': predicted_label}
        answers_list.append(data_dictionary)
        clear_output(wait=True)  
        print(f"Length of dataset_small_A[{label}]: {len(dataset_small_A[label])}")
        print(f"Count: {count}")
        count += 1

datos = answers_list

#WRITE NEW FILE WITH VALIDATE DATA            
file_name = "biomrc/output/biomrc_small_A_scibert_comparison_data.json"            
with open(file_name, 'w') as archivo_json:
    json.dump(datos, archivo_json)

print(f"Length {len(datos)} of comparison results.")
print(f"Saved information in {file_name}") 

#### READ FILE WITH RESULTS ####

In [ ]:
#READ FILE
import json
file_name = "biomrc/output/biomrc_small_A_scibert_comparison_data.json"   

with open(file_name, 'r') as archivo_json:
    biomrc_small_A_scibert_comparison_data = json.load(archivo_json)

#### GET METRICS ####

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

#GET predicted_answers AND true_answers LISTS
predicted_answers = []
true_answers = []
for item in biomrc_small_A_scibert_comparison_data:
    predicted_answers.append(item['predicted_answer'])
    true_answers.append(item['true_answer'])
    
# RECALL AND F1 SCORE
precision, recall, f1, _ = precision_recall_fscore_support(true_answers, predicted_answers, average='weighted', zero_division=0.0)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

In [ ]:
#EXACT MATCH
import re
#Exact Match
em = []
for result in biomrc_small_A_scibert_comparison_data:
    pred = re.sub('[^0-9a-z ]', '', result['predicted_answer'].lower())
    true = re.sub('[^0-9a-z ]', '', result['true_answer'].lower())
    if pred == true:
        em.append(1)
    else:
        em.append(0)

average = sum(em)/len(em)
print("Exact match:")
print(average)
print('\n')

In [ ]:
#ROUGE
from rouge import Rouge
model_out = [ans['predicted_answer'] for ans in biomrc_small_A_scibert_comparison_data]
reference = [ans['true_answer'] for ans in biomrc_small_A_scibert_comparison_data]

clean = re.compile('(?i)[^0-9a-z ]')
model_out = [clean.sub('', text.lower()) for text in model_out]
reference = [clean.sub('', text.lower()) for text in reference]

for i, o in enumerate(model_out):
    if o == "":
        model_out[i] = "vacio"
#print("Model Output:", model_out)
#print("Type of Model Output:", type(model_out))
#print(len(model_out))
#print("\nReference:", reference)
#print("Type of Reference:", type(reference))
#print(len(reference))

rouge = Rouge()
#print("Hola")
result_rouge = rouge.get_scores(model_out, reference)
#print("Hola2")
result_rouge_avg = rouge.get_scores(model_out, reference, avg=true)
scores = rouge.get_scores(model_out, reference)
#print("ROUGE:")
#print(result_rouge)
print("ROUGE AVERAGE:")
print(result_rouge_avg)
indice = 8
print(f"rouge[{indice}]:")
print(model_out[indice], '|', reference[indice], '|', scores[indice]['rouge-1']['f'])
print('\n')

In [ ]:
#F1-SCORE
import numpy as np
#F1-Score
#RECALL = count(match reference y model gram n)/count(reference gram n)
#recall = [print(x) for x in [1,2,3,4,5,6] if x < 5]
list_ref = []
print("F1-SCORE:\n")

#print("Reference:")
for ref in reference:
    split_ref = ref.split()
    list_ref.append(split_ref)

#print(list_ref)

list_model = []
#print("\nModel:")
for model in model_out:
    split_model = model.split()
    list_model.append(split_model)
    
#print(list_model)
#print(list_model[0])
i = 0
list_match = []
for phrase in list_ref:
    count = 0
    for word in phrase:
        if word in list_model[i]:
            count = count + 1
    list_match.append(count)
    i = i+1

list_countref = []
list_countmodel = []
#count ngram
for phrase in list_ref:
    list_countref.append(len(phrase))
for phrase in list_model:
    list_countmodel.append(len(phrase))
#print("\nCount words ref:")
#print(list_countref)
#print("\nCount words model:")
#print(list_countmodel)
#print("\nCount words match:")
#print(list_match)

import numpy as np

match = np.array(list_match)
count_ref = np.array(list_countref)
count_model = np.array(list_countmodel)

array_recall = match/count_ref
array_precision = match/count_model
print("\nRecall array:")
print(array_recall)
print("\nPrecision array:")
print(array_precision)

num = 2*array_precision*array_recall
den = array_precision + array_recall

f1_score = num/den
print("\nF1-Score:")
print(f1_score)
#print(type(f1_score))

# Reemplazar los valores nan por 0.0
my_array_no_nan = np.nan_to_num(f1_score, nan=0.0)
#print(type(my_array_no_nan))
# Convertir el numpy array a una lista
f1_score_list_biomrc = my_array_no_nan.tolist()
#print(type(f1_score_list_baseline))

In [ ]:
filtered_list = [score for score in f1_score_list_biomrc if score <= 1.0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

list_aux = filtered_list #f1_score_list_biomrc

# Histogram
plt.figure(figsize=(10, 6))
plt.hist(list_aux, bins=30, color='green', edgecolor='black')  # Experimenta con el número de bins
# Añadir etiquetas y título
plt.xlabel('F1-Score')
plt.ylabel('Frecuency')
plt.title('BIOMRC scibert_scivocab_uncased - F1-Score Distribution')

# Show
plt.tight_layout()
plt.savefig('biomrc/figures/histogram_biomrc.png')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
list_aux = f1_score_list_biomrc
# Ranges
bins = [0, 0.25, 0.5, 0.75, 1.0]

# Histogram
plt.figure(figsize=(10, 6))
plt.hist(list_aux, bins=bins, color='green', edgecolor='black') 

# Labels and titles
plt.xlabel('F1-Score')
plt.ylabel('Frecuency')
plt.title('BIOMRC scibert_scivocab_uncased - F1-Score Distribution in ranges')

# xticks
plt.xticks(np.arange(0, 1.25, 0.25))

# Show
plt.tight_layout()
plt.savefig('biomrc/figures/histogram_biomrc_ranges.png')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

list_aux = f1_score_list_biomrc

# Ranges
bins = [0, 0.25, 0.5, 0.75, 1.0]

# Frequency
hist, _ = np.histogram(list_aux, bins=bins)

# Labels
labels = [f'{bins[i]:.2f}-{bins[i+1]:.2f}' for i in range(len(bins)-1)]

# Pie Chart
plt.figure(figsize=(8, 8))
plt.pie(hist, labels=labels, autopct='%1.1f%%', startangle=90, colors=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])
plt.title('BIOMRC scibert_scivocab_uncased - Pie chart of F1-Score Distribution in ranges')

# Show
plt.savefig('biomrc/figures/piechart_biomrc.png')
plt.show()